In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
import joblib

In [ ]:
# 2. Load MNIST
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)
print("Data shape:", X.shape)

In [ ]:
# 3. Visualize some digits
plt.figure(figsize=(10,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X[i].reshape(28,28), cmap='gray')
    plt.title(y[i])
    plt.axis('off')
plt.show()

In [ ]:
# 4. Preprocess
X = X / 255.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 5. Define models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='rbf', gamma='scale'),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

In [ ]:
# 6. Train & evaluate
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name} Accuracy: {acc*100:.2f}%")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"{name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

    # Classification report
    print(f"{name} Classification Report:\n")
    print(classification_report(y_test, y_pred))


In [ ]:
# 7. Compare results
best_model_name = max(results, key=results.get)
print(f"\nBest model: {best_model_name} with accuracy {results[best_model_name]*100:.2f}%")
best_model = models[best_model_name]


In [ ]:
# 8. Save best model + scaler
joblib.dump(best_model, "models/mnist_best_model.pkl")
joblib.dump(scaler, "models/mnist_scaler.pkl")

In [ ]:
# 9. Dimensionality reduction visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_test_scaled)
plt.figure(figsize=(10,8))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y_test, cmap='tab10', alpha=0.7)
plt.colorbar()
plt.title("MNIST digits visualized in 2D (PCA)")
plt.show()